<a href="https://www.kaggle.com/code/zittagustavo/xgboost-titanic?scriptVersionId=123049490" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#importando o pandas
import pandas as pd

In [2]:
#lendo os dados de treino
dados=pd.read_csv('/kaggle/input/titanic/train.csv')
#exibindo os dados
dados.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#substituindo variaveis categoricas por categoricas indexadas
gender=pd.get_dummies(dados['Sex'],drop_first=True)
Pclass=pd.get_dummies(dados['Pclass'],drop_first=True)
display(gender.head(),Pclass.head())

,male
0,1
1,0
2,0
3,0
4,1


,2,3
0,0,1
1,0,0
2,0,1
3,0,0
4,0,1


In [4]:
#removendo as colunas antigas do dataframe
dados.drop(['Sex','Pclass'], axis=1, inplace=True)

In [5]:
#colocando as colunas novas no dataframe
dados['Sex']=gender
dados['Pclass']=Pclass.iloc[:, 0]#todas as linhas mas apenas a coluna 1
dados['Pclass1']=Pclass.iloc[:, 1]#todas as linhas mas apenas a coluna 2

In [6]:
#exibindo o novo dataframe
dados.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex,Pclass,Pclass1
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,1,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,0,0,0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,0,0,0
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,1,0,1


In [7]:
#importando biblioteca de graficos
import plotly.express as px
#mapa de calor com base na correlação dos dados
fig = px.imshow(dados.corr(), text_auto=True)
fig.show()

In [8]:
#exibindo as dimensoes que tiveram a maior correlação, tanto positiva quanto negativa
dados[['Parch','Fare','Sex','Pclass','Pclass1']].head()

,Parch,Fare,Sex,Pclass,Pclass1
0,0,7.2500,1,0,1
1,0,71.2833,0,0,0
2,0,7.9250,0,0,1
3,0,53.1000,0,0,0
4,0,8.0500,1,0,1


In [9]:
#como nao sabemos o melhor parametro para o xgboost, passamos varios, entao importamos uma biblioteca que vai testar e pegar o melhor parametro para o nosso modelo

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}

In [10]:
#importando o algoritmo classificador xgboost
import xgboost
#inicializando o xgboost
classifier=xgboost.XGBClassifier()

In [11]:
#aqui estamos usando nossa função para pegar os melhores parametros e 
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [12]:
#criando uma funçao para calcular o tempo
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [13]:
from datetime import datetime
#usando a função de tempo
start_time = timer(None) # timing starts from this point for "start_time" variable
#treinando o modelo  com os parametros aleatorios
random_search.fit(dados[['Parch','Fare','Sex','Pclass','Pclass1']],dados['Survived'])
timer(start_time) # timing ends here for "start_time" variable

Fitting 5 folds for each of 5 candidates, totalling 25 fits

 Time taken: 0 hours 0 minutes and 5.43 seconds.


In [14]:
#vendo os melhores estimadores para nosso modelo
random_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.4,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.2, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [15]:
#vendo os melhores parametros que nos colocamos la em cima
random_search.best_params_

{'min_child_weight': 1,
 'max_depth': 6,
 'learning_rate': 0.3,
 'gamma': 0.2,
 'colsample_bytree': 0.4}

In [16]:
from numpy import nan #importando o nan do numpy
#copiando o output de melhores estimadores e colocando o classificador pra rodar com todos esses parametros
classifier=xgboost.XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=7,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1)

In [17]:
#fazendo a validação cruzada pra score
from sklearn.model_selection import cross_val_score
score=cross_val_score(classifier,dados[['Parch','Fare','Sex','Pclass','Pclass1']],dados['Survived'],cv=10)
score


array([0.76666667, 0.76404494, 0.79775281, 0.82022472, 0.85393258,
       0.79775281, 0.82022472, 0.7752809 , 0.84269663, 0.78651685])

In [18]:
#media dos 10 scores
score.mean()

0.8025093632958802

In [19]:
# agora treinando o modelo com os dados de treino
xgboost=classifier.fit(dados[['Parch','Fare','Sex','Pclass','Pclass1']],dados['Survived'])

In [20]:
#importando e tratando os dados de teste (ja feito anteriormente)
X_test=pd.read_csv('/kaggle/input/titanic/test.csv')
#substituindo variaveis categoricas por categoricas indexadas
gender=pd.get_dummies(X_test['Sex'],drop_first=True)
Pclass=pd.get_dummies(X_test['Pclass'],drop_first=True)
#removendo as colunas antigas do dataframe
X_test.drop(['Sex','Pclass'], axis=1, inplace=True)
#colocando as colunas novas no dataframe
X_test['Sex']=gender
X_test['Pclass']=Pclass.iloc[:, 0]#todas as linhas mas apenas a coluna 1
X_test['Pclass1']=Pclass.iloc[:, 1]#todas as linhas mas apenas a coluna 2
X_test.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex,Pclass,Pclass1
0,892,"Kelly, Mr. James",34.5,0,0,330911,7.8292,NaN,Q,1,0,1
1,893,"Wilkes, Mrs. James (Ellen Needs)",47.0,1,0,363272,7.0000,NaN,S,0,0,1
2,894,"Myles, Mr. Thomas Francis",62.0,0,0,240276,9.6875,NaN,Q,1,1,0
3,895,"Wirz, Mr. Albert",27.0,0,0,315154,8.6625,NaN,S,1,0,1
4,896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",22.0,1,1,3101298,12.2875,NaN,S,0,0,1


In [21]:
#dando predict nos dados de teste e alocando isso em uma variavel
pred=xgboost.predict(X_test[['Parch','Fare','Sex','Pclass','Pclass1']])

In [22]:
#colocando esses dados em um dataframe pa entrega
output = pd.DataFrame({'PassengerId': X_test.PassengerId, 'Survived': pred})
output.to_csv('submission.csv', index=False)
print("Sua entrega foi feita com secesso!")

Sua entrega foi feita com secesso!
